In [1]:
PREPROCESSED_PATH = "E:\Projects\Story-Telling-Using-Show-Attend-and-tell"

def preProBuildWordVocab(sentence_iterator, word_count_threshold=5):
    # borrowed this function from NeuralTalk
    print ('preprocessing word counts and creating vocab based on word count threshold %d' % (word_count_threshold, ))

    word_counts = {}
    nsents = 0

    for sent in sentence_iterator:
        nsents += 1
        tmp_sent = sent.lower().split(' ')
        if '' in tmp_sent:
            tmp_sent.remove('')

        for w in tmp_sent:
           word_counts[w] = word_counts.get(w, 0) + 1

    vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
    print ('filtered words from %d to %d' % (len(word_counts), len(vocab)))

    ixtoword = {}
    ixtoword[0] = '<bos>'
    ixtoword[1] = '<eos>'
    ixtoword[2] = '<pad>'
    ixtoword[3] = '<unk>'

    wordtoix = {}
    wordtoix['<bos>'] = 0
    wordtoix['<eos>'] = 1
    wordtoix['<pad>'] = 2
    wordtoix['<unk>'] = 3

    for idx, w in enumerate(vocab):
        wordtoix[w] = idx + 4
        ixtoword[idx+4] = w

    word_counts['<eos>'] = nsents
    word_counts['<bos>'] = nsents
    word_counts['<pad>'] = nsents
    word_counts['<unk>'] = nsents

    bias_init_vector = np.array([1.0 * word_counts[ ixtoword[i] ] for i in ixtoword])
    bias_init_vector /= np.sum(bias_init_vector) # normalize to frequencies
    bias_init_vector = np.log(bias_init_vector)
    bias_init_vector -= np.max(bias_init_vector) # shift to nice numeric range

    return wordtoix, ixtoword, bias_init_vector


In [2]:


import pickle
import numpy as np
batch_size = 50 # Being support batch_size
num_boxes = 50 # number of Detected regions in each image
feats_dim = 4096 # feature dimensions of each regions
project_dim = 1024 # project the features to one vector, which is 1024 dimensions

sentRNN_lstm_dim = 512 # the sentence LSTM hidden units
sentRNN_FC_dim = 1024 # the fully connected units
wordRNN_lstm_dim = 512 # the word LSTM hidden units
word_embed_dim = 1024 # the learned embedding vectors for the words

S_max = 6
N_max = 30
T_stop = 0.5

n_epochs = 500
learning_rate = 0.0001

open(PREPROCESSED_PATH+'\genomedata\paragraphs_v1.json').read()

img2paragraph = pickle.load(open(PREPROCESSED_PATH+ '\img2paragraph','rb'))
all_sentences = []
for key, paragraph in img2paragraph.items():
    for each_sent in paragraph[1]:
        each_sent.replace(',', ' ,')
        all_sentences.append(each_sent)
word2idx, idx2word, bias_init_vector = preProBuildWordVocab(all_sentences, word_count_threshold=2)
np.save(PREPROCESSED_PATH +'\idx2word_batch', idx2word)

img2paragraph_modify = {}
for img_name, img_paragraph in img2paragraph.items():
    img_paragraph_1 = img_paragraph[1]

    if '' in img_paragraph_1:
        img_paragraph_1.remove('')
    if ' ' in paragraph[1]:
        img_paragraph_1.remove(' ')   
    img_num_sents = len(img_paragraph_1)
    if img_num_sents > S_max:
        img_num_sents = S_max
    img_num_distribution = np.zeros([S_max], dtype=np.int32)
    img_num_distribution[img_num_sents-1:] = 1

    img_captions_matrix = np.ones([S_max, N_max+1], dtype=np.int32) * 2 # zeros([6, 50])
    for idx, img_sent in enumerate(img_paragraph_1):
        # the number of sentences is img_num_sents
        if idx == img_num_sents:
            break

        # because we treat the ',' as a word
        img_sent = img_sent.replace(',', ' ,')

       
        if len(img_sent)>=3 and img_sent[0] == ' ' and img_sent[1] != ' ':
            img_sent = img_sent[1:]
        elif len(img_sent)>=3 and img_sent[0] == ' ' and img_sent[1] == ' ' and img_sent[2] != ' ':
            img_sent = img_sent[2:]

        # Be careful the last part in a sentence, like this:
        # '...world.'
        # '...world. '
        if len(img_sent)>=1 and img_sent[-1] == '.':
            img_sent = img_sent[0:-1]
        elif len(img_sent)>=2 and img_sent[-1] == ' ' and img_sent[-2] == '.':
            img_sent = img_sent[0:-2]

        # Last, we add the <bos> and the <eos> in each sentences
        img_sent = '<bos> ' + img_sent + ' <eos>'

        for idy, word in enumerate(img_sent.lower().split(' ')):
            # because the biggest number of words in a sentence is N_max, here is 50
            if idy == N_max:
                break

            if word in word2idx:
                img_captions_matrix[idx, idy] = word2idx[word]
            else:
                img_captions_matrix[idx, idy] = word2idx['<unk>']

    img2paragraph_modify[str(img_name)] = [img_num_distribution, img_captions_matrix]
with open(PREPROCESSED_PATH+ '\img2paragraph_modify_batch', 'wb') as f:
    pickle.dump(img2paragraph_modify, f)

preprocessing word counts and creating vocab based on word count threshold 2
filtered words from 18926 to 9933


In [11]:
class RegionPooling_HierarchicalRNN():
    def __init__(self, n_words,
                       batch_size,
                       num_boxes,
                       feats_dim,
                       project_dim,
                       sentRNN_lstm_dim,
                       sentRNN_FC_dim,
                       wordRNN_lstm_dim,
                       S_max,
                       N_max,
                       word_embed_dim,
                       bias_init_vector=None):

        self.n_words = n_words
        self.batch_size = batch_size
        self.num_boxes = num_boxes # 50
        self.feats_dim = feats_dim # 4096
        self.project_dim = project_dim # 1024
        self.S_max = S_max # 6
        self.N_max = N_max # 50
        self.word_embed_dim = word_embed_dim # 1024

        self.sentRNN_lstm_dim = sentRNN_lstm_dim # 512 hidden size
        self.sentRNN_FC_dim = sentRNN_FC_dim # 1024 in fully connected layer
        self.wordRNN_lstm_dim = wordRNN_lstm_dim # 512 hidden size

        # word embedding, parameters of embedding
        # embedding shape: n_words x wordRNN_lstm_dim
        with tf.device('/cpu:0'):
            self.Wemb = tf.Variable(tf.random_uniform([n_words, word_embed_dim], -0.1, 0.1), name='Wemb')
        #self.bemb = tf.Variable(tf.zeros([word_embed_dim]), name='bemb')

        # regionPooling_W shape: 4096 x 1024
        # regionPooling_b shape: 1024
        self.regionPooling_W = tf.Variable(tf.random_uniform([feats_dim, project_dim], -0.1, 0.1), name='regionPooling_W')
        self.regionPooling_b = tf.Variable(tf.zeros([project_dim]), name='regionPooling_b')

        # sentence LSTM
        self.sent_LSTM = tf.nn.rnn_cell.BasicLSTMCell(sentRNN_lstm_dim, state_is_tuple=True)

        # logistic classifier
        self.logistic_Theta_W = tf.Variable(tf.random_uniform([sentRNN_lstm_dim, 2], -0.1, 0.1), name='logistic_Theta_W')
        self.logistic_Theta_b = tf.Variable(tf.zeros(2), name='logistic_Theta_b')

        # fc1_W: 512 x 1024, fc1_b: 1024
        # fc2_W: 1024 x 1024, fc2_b: 1024
        self.fc1_W = tf.Variable(tf.random_uniform([sentRNN_lstm_dim, sentRNN_FC_dim], -0.1, 0.1), name='fc1_W')
        self.fc1_b = tf.Variable(tf.zeros(sentRNN_FC_dim), name='fc1_b')
        self.fc2_W = tf.Variable(tf.random_uniform([sentRNN_FC_dim, 1024], -0.1, 0.1), name='fc2_W')
        self.fc2_b = tf.Variable(tf.zeros(1024), name='fc2_b')

        # word LSTM
        self.word_LSTM = tf.nn.rnn_cell.BasicLSTMCell(wordRNN_lstm_dim, state_is_tuple=True)
        self.word_LSTM = tf.nn.rnn_cell.MultiRNNCell([self.word_LSTM,self.word_LSTM], state_is_tuple=True)
        self.word_LSTM = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(wordRNN_lstm_dim,state_is_tuple=True) for _ in range(2)])

        self.embed_word_W = tf.Variable(tf.random_uniform([wordRNN_lstm_dim, n_words], -0.1,0.1), name='embed_word_W')
        if bias_init_vector is not None:
            self.embed_word_b = tf.Variable(bias_init_vector.astype(np.float32), name='embed_word_b')
            self.embed_word_b = tf.Variable(tf.zeros([n_words]), name='embed_word_b')
    def build_model(self):
       
        feats = tf.placeholder(tf.float32, [self.batch_size, self.num_boxes, self.feats_dim])
        tmp_feats = tf.reshape(feats, [-1, self.feats_dim])

      
        project_vec_all = tf.matmul(tmp_feats, self.regionPooling_W) + self.regionPooling_b
        project_vec_all = tf.reshape(project_vec_all, [self.batch_size, 50, self.project_dim])
        project_vec = tf.reduce_max(project_vec_all, reduction_indices=1)

        
        num_distribution = tf.placeholder(tf.int32, [self.batch_size, self.S_max])

        captions = tf.placeholder(tf.int32, [self.batch_size, self.S_max, self.N_max+1])
        captions_masks = tf.placeholder(tf.float32, [self.batch_size, self.S_max, self.N_max+1])

        
        sent_state = self.sent_LSTM.zero_state(batch_size=self.batch_size, dtype=tf.float32)
       

        probs = []
        loss = 0.0
        loss_sent = 0.0
        loss_word = 0.0
        lambda_sent = 5.0
        lambda_word = 1.0

        print ('Start build model:')
        
        for i in range(0, self.S_max):
            print("naveen")
            if i > 0:
                tf.get_variable_scope().reuse_variables()
            with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
                sent_output, sent_state = self.sent_LSTM(project_vec, sent_state)

            with tf.name_scope('fc1'):
                hidden1 = tf.nn.relu( tf.matmul(sent_output, self.fc1_W) + self.fc1_b )
            with tf.name_scope('fc2'):
                sent_topic_vec = tf.nn.relu( tf.matmul(hidden1, self.fc2_W) + self.fc2_b )

           
            sentRNN_logistic_mu = tf.nn.xw_plus_b( sent_output, self.logistic_Theta_W, self.logistic_Theta_b )
            sentRNN_label = tf.stack([ 1 - num_distribution[:, i], num_distribution[:, i] ])
            sentRNN_label = tf.transpose(sentRNN_label)
            # https://github.com/ibab/tensorflow-wavenet/issues/223
            sentRNN_loss = tf.nn.softmax_cross_entropy_with_logits(labels=sentRNN_label, logits=sentRNN_logistic_mu)
            sentRNN_loss = tf.reduce_sum(sentRNN_loss)/self.batch_size
            loss += sentRNN_loss * lambda_sent
            loss_sent += sentRNN_loss

       
            topic = tf.nn.rnn_cell.LSTMStateTuple(sent_topic_vec[:, 0:512], sent_topic_vec[:, 512:])
            word_state = (topic, topic)
            for j in range(0, self.N_max):
                if j > 0:
                    tf.get_variable_scope().reuse_variables()

                with tf.device('/cpu:0'):
                    with tf.variable_scope('Wemb',reuse=tf.AUTO_REUSE):
                        current_embed = tf.nn.embedding_lookup(self.Wemb, captions[:, i, j])
                    
                with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
                    # pdb.set_trace()
                    word_output, word_state = self.word_LSTM(current_embed, word_state)

                labels = tf.reshape(captions[:, i, j+1], [-1, 1])
                indices = tf.reshape(tf.range(0, self.batch_size, 1), [-1, 1])
                concated = tf.concat([indices, labels], 1)
                onehot_labels = tf.sparse_to_dense(concated, tf.stack([self.batch_size, self.n_words]), 1.0, 0.0)

                logit_words = tf.nn.xw_plus_b(word_output[:], self.embed_word_W, self.embed_word_b)
                cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit_words, labels=onehot_labels)
                cross_entropy = cross_entropy * captions_masks[:, i, j]
                loss_wordRNN = tf.reduce_sum(cross_entropy) / self.batch_size
                loss += loss_wordRNN * lambda_word
                loss_word += loss_wordRNN

        return feats, num_distribution, captions, captions_masks, loss, loss_sent, loss_word

    def generate_model(self):
        feats = tf.placeholder(tf.float32, [1, self.num_boxes, self.feats_dim])
        tmp_feats = tf.reshape(feats, [-1, self.feats_dim])

        project_vec_all = tf.matmul(tmp_feats, self.regionPooling_W) + self.regionPooling_b
        project_vec_all = tf.reshape(project_vec_all, [1, 50, self.project_dim])
        project_vec = tf.reduce_max(project_vec_all, reduction_indices=1)

        sent_state = self.sent_LSTM.zero_state(batch_size=1, dtype=tf.float32)

        # save the generated paragraph to list, here I named generated_sents
        generated_paragraph = []

        # pred
        pred_re = []

        T_stop = tf.constant(0.5)

        # Start build the generation model
        print ('Start build the generation model: ')

     
        for i in range(0, self.S_max):
            if i > 0:
                tf.get_variable_scope().reuse_variables()
            sent_output, sent_state = self.sent_LSTM(project_vec, sent_state)

        
            with tf.name_scope('fc1'):
                hidden1 = tf.nn.relu( tf.matmul(sent_output, self.fc1_W) + self.fc1_b )
            with tf.name_scope('fc2'):
                sent_topic_vec = tf.nn.relu( tf.matmul(hidden1, self.fc2_W) + self.fc2_b )

            sentRNN_logistic_mu = tf.nn.xw_plus_b(sent_output, self.logistic_Theta_W, self.logistic_Theta_b)
            pred = tf.nn.softmax(sentRNN_logistic_mu)
            pred_re.append(pred)

           
            generated_sent = []

          
            topic = tf.nn.rnn_cell.LSTMStateTuple(sent_topic_vec[:, 0:512], sent_topic_vec[:, 512:])
            word_state = (topic, topic)
            # word RNN, unrolled to N_max time steps
            for j in range(0, self.N_max):
                if j > 0:
                    tf.get_variable_scope().reuse_variables()

                if j == 0:
                    with tf.device('/cpu:0'):
                    # get word embedding of BOS (index = 0)
                        current_embed = tf.nn.embedding_lookup(self.Wemb, tf.zeros([1], dtype=tf.int64))

                with tf.variable_scope('word_LSTM'):
                    word_output, word_state = self.word_LSTM(current_embed, word_state)

                logit_words = tf.nn.xw_plus_b(word_output, self.embed_word_W, self.embed_word_b)
                max_prob_index = tf.argmax(logit_words, 1)[0]
                generated_sent.append(max_prob_index)

                with tf.device('/cpu:0'):
                    current_embed = tf.nn.embedding_lookup(self.Wemb, max_prob_index)
                    current_embed = tf.expand_dims(current_embed, 0)

            generated_paragraph.append(generated_sent)
        return feats, generated_paragraph, pred_re, sent_topic_vec
    

In [ ]:
import h5py
import os
import tensorflow as tf
import random
import matplotlib.pyplot as plt
def train():
    ##############################################################################
    # some preparing work
    ##############################################################################
    model_path = PREPROCESSED_PATH+'\models_batch\\'
    train_feats_path = PREPROCESSED_PATH+'\genomedata\im2p_train_output.h5'
    if os.path.isfile(train_feats_path) and os.access(train_feats_path, os.R_OK):
        train_output_file = h5py.File(train_feats_path, 'r')
    else :
        print("Either file is missing or is not readable")
        
    train_feats = train_output_file.get('feats')
    train_imgs_full_path_lists = open(PREPROCESSED_PATH+'\imgs_train_path.txt').read().splitlines()
    train_imgs_names = map(lambda x: os.path.basename(x).split('.')[0], train_imgs_full_path_lists)
   # print(list(train_imgs_names))
    model = RegionPooling_HierarchicalRNN(n_words = len(word2idx),
                                          batch_size = batch_size,
                                          num_boxes = num_boxes,
                                          feats_dim = feats_dim,
                                          project_dim = project_dim,
                                          sentRNN_lstm_dim = sentRNN_lstm_dim,
                                          sentRNN_FC_dim = sentRNN_FC_dim,
                                          wordRNN_lstm_dim = wordRNN_lstm_dim,
                                          S_max = S_max,
                                          N_max = N_max,
                                          word_embed_dim = word_embed_dim,
                                          bias_init_vector = bias_init_vector)
    tf_feats, tf_num_distribution, tf_captions_matrix, tf_captions_masks, tf_loss, tf_loss_sent, tf_loss_word = model.build_model()
    with tf.Session() as sess:
        print ("start creating session")
        # saver = tf.train.Saver(max_to_keep=500, write_version=1)
        with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
            train_op = tf.train.AdamOptimizer(learning_rate).minimize(tf_loss)
        tf.global_variables_initializer().run()

        # when you want to train the model from the previously saved model
        # how many models we want to save
        saver = tf.train.Saver(max_to_keep=150)
        # before TF v1, by cxp
        # new_saver = tf.train.import_meta_graph('./models_batch/model-250.meta')
        # print "meta file imported"
        # model_file = tf.train.latest_checkpoint('./models_batch')  
        # new_saver.restore(sess, model_file)
        
        try:
            saver.restore(sess, './models_batch/model-20')
            print ("pretrained model loaded successfully")
        except:
            print ("fail to load pretrained model")
            pass
        
        all_vars = tf.trainable_variables()

        # open a loss file to record the loss value
        loss_fd = open('loss_batch.txt', 'a')
        img2idx = {}
        for idx, img in enumerate(train_imgs_names):
            img2idx[img] = idx

        loss_to_draw = []
        plt_save_dir = './loss_imgs'
        
        for epoch in range(0, n_epochs):
            loss_to_draw_epoch = []
            random.shuffle(list(train_imgs_names))

            for start, end in zip(range(0, len(list(train_imgs_names)), batch_size),
                                  range(batch_size, len(list(train_imgs_names)), batch_size)):

                start_time = time.time()

                img_name = train_imgs_names[start:end]
                current_feats_index = map(lambda x: img2idx[x], img_name)
                current_feats = np.asarray( map(lambda x: train_feats[x], current_feats_index) )

                current_num_distribution = np.asarray( map(lambda x: img2paragraph_modify[x][0], img_name) )
                current_captions_matrix = np.asarray( map(lambda x: img2paragraph_modify[x][1], img_name) )

                current_captions_masks = np.zeros( (current_captions_matrix.shape[0], current_captions_matrix.shape[1], current_captions_matrix.shape[2]) )
                nonzeros = np.array( map(lambda each_matrix: np.array( map(lambda x: (x != 2).sum() + 1, each_matrix ) ), current_captions_matrix ) )
                for i in range(batch_size):
                    for ind, row in enumerate(current_captions_masks[i]):
                        row[:(nonzeros[i, ind]-1)] = 1

                _, loss_val, loss_sent, loss_word= sess.run(
                                    [train_op, tf_loss, tf_loss_sent, tf_loss_word],
                                    feed_dict={
                                               tf_feats: current_feats,
                                               tf_num_distribution: current_num_distribution,
                                               tf_captions_matrix: current_captions_matrix,
                                               tf_captions_masks: current_captions_masks
                                    })

                loss_to_draw_epoch.append(loss_val)

                if idx % 1000 == 0:
                    print ('idx: ', start, ' Epoch: ', epoch, ' loss: ', loss_val, ' loss_sent: ', loss_sent, ' loss_word: ', loss_word, \
                          ' Time cost: ', str((time.time() - start_time)))
                loss_fd.write('epoch ' + str(epoch) + ' loss ' + str(loss_val))
            
            loss_to_draw.append(np.mean(loss_to_draw_epoch))

            if np.mod(epoch, 10) == 0:
                # draw loss curve every 10 epochs
                plt_save_img_name = str(epoch) + '.png'
                plt.plot(range(len(loss_to_draw)), loss_to_draw, color='g')
                plt.grid(True)
                plt.savefig(os.path.join(plt_save_dir, plt_save_img_name))
            
                # save weights every 20 epochs
                if np.mod(epoch, 20) == 0:
                    print ("Epoch ", epoch, " is done. Saving the model ...")
                    saver.save(sess, os.path.join(model_path, 'model'), global_step=epoch, write_meta_graph=False)
        loss_fd.close()
    

    

In [ ]:
train()

Start build model:
naveen
naveen
naveen
naveen
naveen
naveen
start creating session
INFO:tensorflow:Restoring parameters from ./models_batch/model-20
fail to load pretrained model


C:\Users\naveen\AppData\Roaming\Python\Python35\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\naveen\AppData\Roaming\Python\Python35\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch  0  is done. Saving the model ...
Epoch  20  is done. Saving the model ...
Epoch  40  is done. Saving the model ...
Epoch  60  is done. Saving the model ...
Epoch  80  is done. Saving the model ...
Epoch  100  is done. Saving the model ...
Epoch  120  is done. Saving the model ...
Epoch  140  is done. Saving the model ...
Epoch  160  is done. Saving the model ...
Epoch  180  is done. Saving the model ...
Epoch  200  is done. Saving the model ...
Epoch  220  is done. Saving the model ...
Epoch  240  is done. Saving the model ...
Epoch  260  is done. Saving the model ...
Epoch  280  is done. Saving the model ...
Epoch  300  is done. Saving the model ...
Epoch  320  is done. Saving the model ...
Epoch  340  is done. Saving the model ...


In [ ]:
def test():
    start_time = time.time()
    # change the model path according to your environment
    model_path = './models_batch/model-500'

    # It's very important to use Pandas to Series this idx2word dict
    # After this operation, we can use list to extract the word at the same time
    idx2word = pd.Series(np.load('./data/idx2word_batch.npy').tolist())

    test_feats_path = './data/im2p_test_output.h5'
    test_output_file = h5py.File(test_feats_path, 'r')
    test_feats = test_output_file.get('feats')

    test_imgs_full_path_lists = open('./densecap/imgs_test_order.txt').read().splitlines()
    test_imgs_names = map(lambda x: os.path.basename(x).split('.')[0], test_imgs_full_path_lists)

    # n_words, batch_size, num_boxes, feats_dim, project_dim, sentRNN_lstm_dim, sentRNN_FC_dim, wordRNN_lstm_dim, S_max, N_max
    test_model = RegionPooling_HierarchicalRNN(n_words = len(word2idx),
                                               batch_size = batch_size,
                                               num_boxes = num_boxes,
                                               feats_dim = feats_dim,
                                               project_dim = project_dim,
                                               sentRNN_lstm_dim = sentRNN_lstm_dim,
                                               sentRNN_FC_dim = sentRNN_FC_dim,
                                               wordRNN_lstm_dim = wordRNN_lstm_dim,
                                               S_max = S_max,
                                               N_max = N_max,
                                               word_embed_dim = word_embed_dim,
                                               bias_init_vector = bias_init_vector)

    tf_feats, tf_generated_paragraph, tf_pred_re, tf_sent_topic_vectors = test_model.generate_model()
    sess = tf.InteractiveSession()

    saver = tf.train.Saver()
    saver.restore(sess, model_path)

    img2idx = {}
    for idx, img in enumerate(test_imgs_names):
        img2idx[img] = idx

    test_fd = open('HRNN_results.txt', 'w')
    for idx, img_name in enumerate(test_imgs_names):
        print (idx, img_name)
        test_fd.write(img_name + '\n')

        each_paragraph = []
        current_paragraph = ""

        current_feats_index = img2idx[img_name]
        current_feats = test_feats[current_feats_index]
        current_feats = np.reshape(current_feats, [1, 50, 4096])

        generated_paragraph_indexes, pred, sent_topic_vectors = sess.run(
                                                                         [tf_generated_paragraph, tf_pred_re, tf_sent_topic_vectors],
                                                                         feed_dict={
                                                                             tf_feats: current_feats
                                                                         })

        #generated_paragraph = idx2word[generated_paragraph_indexes]
        for sent_index in generated_paragraph_indexes:
            each_sent = []
            for word_index in sent_index:
                each_sent.append(idx2word[word_index])
            each_paragraph.append(each_sent)

        for idx, each_sent in enumerate(each_paragraph):
            # if the current sentence is the end sentence of the paragraph
            # According to the probability distribution:
            # CONTINUE: [1, 0]
            # STOP    : [0, 1]
            # So, if the first item of pred is less than the T_stop
            # the generation process is break
            if pred[idx][0][0] <= T_stop:
                break
            current_sent = ''
            for each_word in each_sent:
                current_sent += each_word + ' '
            current_sent = current_sent.replace('<eos> ', '')
            current_sent = current_sent.replace('<pad> ', '')
            current_sent = current_sent + '.'
            current_sent = current_sent.replace(' .', '.')
            current_sent = current_sent.replace(' ,', ',')
            current_paragraph +=current_sent
            if idx != len(each_paragraph) - 1:
                current_paragraph += ' '

        test_fd.write(current_paragraph + '\n')
    test_fd.close()
    print ("Time cost: " + str(time.time()-start_time))
